In [2]:
import pandas as pd
import html5lib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [3]:
def load_raw():
    dataframes=[]
    for k in tqdm(range(2001,2019)):
        for i in range(1,13):
            df = pd.read_html('./data/'+str(k)+'Y_'+str(i)+'M.xls',encoding='utf-8',header=0)
            dataframes.append(df[0])
    return dataframes

In [4]:
dataframes=load_raw()

100%|██████████| 18/18 [00:36<00:00,  4.18s/it]


In [5]:
df_nans=[]
for df in dataframes:
    df_nan=df[df.isna().any(1)]
    df_nan=df_nan.dropna(axis='columns')
    # 2007/12 2013/12 change
    df_nan.rename(columns={'類型代號':"基金名稱","基金統編":"基金規模","基金規模":"本月定時定額扣款筆數","單位淨值":"本月定時定額扣款人數","本月定時定額扣款筆數":"本月定時定額扣款金額"})
    df_nans.append(df_nan)

In [6]:
df_corrects=[]
df_filter = []
# for df in dataframes:
#     df_correct = df[~df.isna().any(1)]
#     df_corrects.append(df_correct)
for df in tqdm(dataframes):
    df.rename(columns={"基金規模(台幣)":"基金規模","單位淨值(台幣)":"單位淨值","本月定時定額扣款金額(台幣)":"本月定時定額扣款金額"}, inplace=True)
    df_correct = df[~df.isna().any(1)]
    df_filter = []
    for name in df_correct['基金名稱']:
        if ("中國" in name or "俄羅斯" in name or "巴西" in name or "印度" in name or "醫療" in name or "生技" in name) :
            if("中國信託" in name):
                continue
            filter1 = (df_correct['類型代號'] == "AA2")
            filter2 = (df_correct['類型代號'] == "2")
            filter3 = (df_correct['基金名稱'] == name)
            df_filter.append(df_correct[((filter1 | filter2) & filter3)])
    df_corrects.append(pd.concat(df_filter))
df_corrects[0]# 0=2001

100%|██████████| 216/216 [00:25<00:00,  1.76it/s]


,類型代號,基金統編,基金名稱,基金規模,單位淨值,本月定時定額扣款筆數,本月定時定額扣款人數,本月定時定額扣款金額
86,2,97990289,元大中國基金,436898521,6.54,8,8,63000


In [7]:
def plot_image(id):
    navs=[]
    money=[]
    num=[]
    people=[]
    scales=[]
    id = str(id)
    id = id.zfill(8)
    for i in range(0,2*12):
        try:
            navs.append(df_corrects[i][df_corrects[i]['基金統編']==id]['單位淨值'].values[0])
            money.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款金額'].values[0])
            num.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
            people.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
            scales.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金規模'].values[0])
        except:
            pass
    plt.plot(navs)
    plt.title('Unit net value')
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP value')
    plt.plot(money)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Number of records')
    plt.plot(num)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Number of people')
    plt.plot(people)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    plt.title('SIP Scale')
    plt.plot(scales)
    plt.ylabel('Value')
    plt.xlabel('Months')
    plt.show()
    

## Function of calculating rising or falling

In [8]:
def monthFilter(month, length, data):
    subData = np.array(data[(month - (month % 6)) : (month + (5-(month % 6)) + 1)])
    value=(data[month] - np.mean(subData)) / np.std(subData)
    if(value> 1 or value < -1):
        return value
    else:
        return 0

## This function will get various people,aliveList,variations

In [9]:
def preprocessing():
    id_list = []
    id_temp = []
    aliveList=[]
    uid=[]
    upeople=[]
    peopleChinaList = []
    peopleIndiaList = []
    peopleBrazilList = []
    peopleRussiaList = []
    peopleMedicalList = []
    markMonthChinaList =[]
    markMonthIndiaList = []
    markMonthBrazilList = []
    markMonthRussiaList = []
    markMonthlMedicalList = []
    MonthChinaValue=[]
    MonthIndiaValue=[]
    MonthRussiaValue=[]
    MonthBrazilValue=[]
    for df in df_corrects:
        for filter_id in df['基金統編']:
#             id = str(filter_id)
#             id = id.zfill(8)
            id_temp.append(filter_id)
    id_list = list(set(id_temp))
    for id in id_list:
            isAlive = True
            navs=[]
            money=[]
            num=[]
            people=[]
            scales=[]
            name = ''
            isSaveList = []
            for i in range((17-3)*12 + 4,17*12 + 4):
                try:
                    name = df_corrects[i][df_corrects[i]['基金統編']==id]['基金名稱'].values[0]
                    navs.append(df_corrects[i][df_corrects[i]['基金統編']==id]['單位淨值'].values[0])
                    money.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款金額'].values[0])
                    num.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款筆數'].values[0])
                    people.append(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款人數'].values[0])
                    scales.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金規模'].values[0])
                    if(df_corrects[i][df_corrects[i]['基金統編']==id]['本月定時定額扣款人數'].values[0]) == 0 :
                        isAlive = False
                    if(i == 17*12 + 3 and isAlive):
                        aliveList.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金名稱'].values[0])
                        uid.append(df_corrects[i][df_corrects[i]['基金統編']==id]['基金統編'].values[0])
                
                except:
                    isAlive = False
                    navs.append(0)
                    money.append(0)
                    num.append(0)
                    people.append(0)
                    scales.append(0)
                    break
            funding={}
            funding
            if(isAlive):
                for i in range(0, len(people)):
                    if("中國" in name):
                        try:
                            peopleChinaList[i] += people[i]
                        except:
                            peopleChinaList.append(people[i])
                    elif("印度" in name):
                        try:
                            peopleIndiaList[i] += people[i]
                        except:
                            peopleIndiaList.append(people[i])
                    elif("巴西" in name):
                        try:
                            peopleBrazilList[i] += people[i]
                        except:
                            peopleBrazilList.append(people[i])
                    elif("俄羅斯" in name):
                        try:
                            peopleRussiaList[i] += people[i]
                        except:
                            peopleRussiaList.append(people[i])
                    else:
                        try:
                            peopleMedicalList[i] += people[i] 
                        except:
                            peopleMedicalList.append(people[i])
#                 plt.plot(navs)
#                 plt.title(id + 10*' ' +'Unit net value')
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP value')
#                 plt.plot(money)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Number of records')
#                 plt.plot(num)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Number of people')
#                 plt.plot(people)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()
#                 plt.title(id + 10*' ' +'SIP Scale')
#                 plt.plot(scales)
#                 plt.ylabel('Value')
#                 plt.xlabel('Months')
#                 plt.show()

    # Calculate people and value
    
    china={}
    for i in range(0, len(peopleChinaList)):
        value=monthFilter(i, 6, peopleChinaList)
        if value!=0:
            markMonthChinaList.append(i)
            MonthChinaValue.append(value)
    china={
        'Month_Index':markMonthChinaList,
        'Value':MonthChinaValue
    }
    brazil={}
    for i in range(0, len(peopleBrazilList)):
        value=monthFilter(i, 6, peopleBrazilList)
        if value!=0:
            markMonthBrazilList.append(i)
            MonthBrazilValue.append(value)
    brazil={
        'Month_Index':markMonthBrazilList,
        'Value':MonthBrazilValue
    }
    for i in range(0, len(peopleIndiaList)):
        value=monthFilter(i, 6, peopleIndiaList)
        if value!=0:
            markMonthIndiaList.append(i)
            MonthIndiaValue.append(value)
    india={
        'Month_Index':markMonthBrazilList,
        'Value':MonthBrazilValue
    }
    for i in range(0, len(peopleRussiaList)):
        value=monthFilter(i, 6, peopleIndiaList)
        if value!=0:
            markMonthIndiaList.append(i)
            MonthIndiaValue.append(value)
    russia={
        'Month_Index':markMonthRussiaList,
        'Value':MonthRussiaValue
    }
    variation={
        'China':pd.DataFrame(china),
        'Brazil':pd.DataFrame(brazil),
        'India':pd.DataFrame(india),
        'Russia':pd.DataFrame(russia)
    }
    
#     for i in range(0, len(peopleMedicalList)):
#         if(monthFilter(i, 6, peopleMedicalList)):
#             markMonthlMedicalList.append(i)
#     plt.title('SIP Number of people in Medical')
#     plt.plot(peopleMedicalList)
#     plt.ylabel('Value')
#     plt.xlabel('Months')
#     plt.show()
    china_df=pd.DataFrame(china)
    dict={}
    start_date = "2016/04" 
    stop_date="2019/03"
    start=datetime.strptime(start_date,"%Y/%m")
    stop=datetime.strptime(stop_date,"%Y/%m")
    arr_smonth=[]
    while start<=stop:
        arr_smonth.append(start.strftime('%Y/%m')) # Return a string representing the date and time
        start = start +  relativedelta(months=1)
    percentage_c=[]
    percentage_b=[]
    percentage_r=[]
    percentage_i=[]
    for i in range(len(peopleChinaList)):
        total=peopleChinaList[i]+peopleIndiaList[i]+peopleRussiaList[i]+peopleBrazilList[i]
        percentage_c.append((peopleChinaList[i]/total)*100)
        percentage_b.append((peopleBrazilList[i]/total)*100)
        percentage_r.append((peopleRussiaList[i]/total)*100)
        percentage_i.append((peopleIndiaList[i]/total)*100)
        
        
    
    dict={
        'Date':arr_smonth,
        'China':peopleChinaList,
        'India':peopleIndiaList,
        'Russia':peopleRussiaList,
        'Brazil':peopleBrazilList,
        'China%':percentage_c,
        'India%':percentage_i,
        'Russia%':percentage_r,
        'Brazil%':percentage_b
    }
    alive={
        "name":aliveList,
        "uid":uid
    }
    
 
    peopleDF=pd.DataFrame(dict)
    aliveDF=pd.DataFrame(alive)
    return aliveList,aliveDF,variation,peopleDF



In [10]:
aliveList,aliveDF,varation ,people= preprocessing()
varation['China']

,Month_Index,Value
0,0,-1.856616
1,9,-2.054153
2,12,1.923390
3,17,-1.151120
4,20,-1.889533
5,22,1.122276
6,24,-1.470169
7,29,1.457625
8,33,-1.806295


In [41]:
people

,Date,China,India,Russia,Brazil,China%,India%,Russia%,Brazil%
0,2016/04,19451,2267,2104,3244,71.865071,8.375822,7.773590,11.985517
1,2016/05,22223,2247,2058,3210,74.729303,7.555989,6.920438,10.794270
2,2016/06,22482,2231,2040,3153,75.175550,7.460041,6.821374,10.543035
3,2016/07,22306,2442,1965,2971,75.144859,8.226654,6.619728,10.008759
4,2016/08,20813,2413,1881,2754,74.702990,8.660852,6.751373,9.884785
5,2016/09,21304,2360,1811,2597,75.890567,8.406954,6.451268,9.251211
6,2016/10,21493,2390,1738,2490,76.457614,8.502010,6.182633,8.857743
7,2016/11,21451,2314,1710,2375,77.023339,8.308797,6.140036,8.527828
8,2016/12,20909,2231,1663,2222,77.369103,8.255319,6.153562,8.222017
9,2017/01,17905,2046,1535,2062,76.036181,8.688636,6.518600,8.756582


In [12]:
newAliveList = []
for name in aliveList:
    if("新臺幣" in name):
        name = name.replace("新臺幣", "台幣")
    if("新台幣" in name):
        name = name.replace("新台幣", "台幣")
    if("台幣類型" in name):
        name = name.replace("台幣類型", "台幣")
    if("N" in name):
        name = name.replace("N", "N類型")
    if("-N類型" in name):
        name = name.replace("-N類型", "")
        name = name.replace("基金", "基金N類型")
    if("級別" in name):
        if("台幣" in name):
            name = name.replace("台幣級別", "(台幣)")
        if("人民幣" in name):
            name = name.replace("人民幣級別", "(人民幣)")
        if("美元" in name):
            name = name.replace("美元級別", "(美元)")
    if("-" in name):
        name = name.replace("-", "(", 1) + ")"
    if("人民幣類型" in name):
        name = name.replace("類型", "")
    if("兆豐國際中國A股基金(美金)" in name):
        name = name.replace("美金", "美元")
    if("安聯中國策略基金" in name):
        name = name + "(台幣)"
    if("台新中國精選中小基金" in name):
        name = name + "(台幣)"
    if("摩根中國A股基金" in name):
        if("摩根中國A股基金(美元)" in name):
            pass
        else:
            name = name + "(台幣)"
    if("瀚亞中國" in name):
        name = name.replace("中國", "中國A股")
    if("野村新興傘型基金之大俄羅斯基金" in name):
        name = name.replace("野村新興傘型基金之大俄羅斯基金", "野村大俄羅斯基金")
    if("野村雙印傘型基金之印度潛力基金" in name):
        name = name.replace("野村雙印傘型基金之印度潛力基金", "野村印度潛力基金")
    newAliveList.append(name)
# newAliveList

In [13]:
newAliveList

['日盛中國內需動力基金',
 '野村大俄羅斯基金',
 '群益全球關鍵生技基金(台幣)',
 '野村中國機會基金',
 '第一金中國世紀基金(台幣)',
 '野村巴西基金',
 '野村全球生技醫療基金',
 '復華中國新經濟A股基金(台幣)',
 '德信中國精選成長基金',
 '安聯中國東協基金',
 '摩根中國亮點基金',
 '野村印度潛力基金',
 '保德信全球醫療生化基金(台幣)',
 '安聯中國策略基金(台幣)',
 '國泰中國內需增長基金(台幣)',
 '兆豐國際中國A股基金(台幣)',
 '台新印度基金',
 '瀚亞巴西基金',
 '群益中國新機會基金(台幣)',
 '安聯全球生技趨勢基金(台幣)',
 '兆豐國際中國A股基金(美元)',
 '元大印度基金',
 '台新中國精選中小基金(台幣)']

In [14]:
# df_corrects[172][df_corrects[172]['基金名稱'] =='瀚亞巴西基金']['本月定時定額扣款人數']

In [15]:
aliveDF

,name,uid
0,日盛中國內需動力基金,26323737
1,野村新興傘型基金之大俄羅斯基金,10306992
2,群益全球關鍵生技基金-新臺幣,38486708A
3,野村中國機會基金,25530596
4,第一金中國世紀基金-新臺幣,25620512A
5,野村巴西基金,25652008
6,野村全球生技醫療基金,48844514
7,復華中國新經濟A股基金-新臺幣,42303511A
8,德信中國精選成長基金,25689835
9,安聯中國東協基金,25680150


In [16]:
def country_classifier(newAliveList):
    country=dict()
    china_id=[]
    china_n=[]
    india_id=[]
    india_n=[]
    brazil_id=[]
    brazil_n=[]
    russia_id=[]
    russia_n=[]
    for funding,uid in zip(newAliveList['name'],newAliveList['uid']):
        if '中國' in funding:
            china_id.append(uid)
            china_n.append(funding)
        elif '巴西' in funding:
            brazil_id.append(uid)
            brazil_n.append(funding)
        elif '印度' in funding:
            india_id.append(uid)
            india_n.append(funding)
        elif '俄羅斯' in funding:
            russia_id.append(uid)
            russia_n.append(funding)
    country_id={
        'China':china_id,
        'Brazil':brazil_id,
        'India':india_id,
        'Russia':russia_id
    }
    country_n={
        'China':china_n,
        'Brazil':brazil_n,
        'India':india_n,
        'Russia':russia_n
        
    }
    return country_n,country_id

In [17]:
result_n,result_id=country_classifier(aliveDF)

In [18]:
def get_funding(dataframe):
    start_date = "2016/04" 
    stop_date="2019/03"
    start=datetime.strptime(start_date,"%Y/%m")
    stop=datetime.strptime(stop_date,"%Y/%m")
    arr_smonth=[]
    while start<=stop:
        arr_smonth.append(start.strftime('%Y/%m')) 
        start = start +  relativedelta(months=1)
    sip_people={}
    sip_people['Date']=arr_smonth
    for name,item in zip(dataframe['name'],dataframe['uid']):
        arr=[]
        for i in range(14*12+4,17*12+4):
            a=df_corrects[i][df_corrects[i]['基金統編']==item]['本月定時定額扣款人數'].values[0]
            arr.append(a)
        sip_people[item]=arr        
    return pd.DataFrame(sip_people)

In [19]:
result_n['China']

['日盛中國內需動力基金',
 '野村中國機會基金',
 '第一金中國世紀基金-新臺幣',
 '復華中國新經濟A股基金-新臺幣',
 '德信中國精選成長基金',
 '安聯中國東協基金',
 '摩根中國亮點基金',
 '安聯中國策略基金',
 '國泰中國內需增長基金台幣級別',
 '兆豐國際中國A股基金(台幣)',
 '群益中國新機會基金-新臺幣',
 '兆豐國際中國A股基金(美金)',
 '台新中國精選中小基金']

In [24]:
result_id['China']

['25589337',
 '25530596',
 '42303511A',
 '26323737',
 '38524463A',
 '25680150',
 '38592074A',
 '25689835',
 '25691213',
 '26317735A',
 '25620512A',
 '38592074B',
 '25627402']

In [32]:
def get_moneys(ids):
    sip_moneys={}
    for item in ids:
        sip_money=[]
        for i in range(14*12+4,17*12+4):
                    a=df_corrects[i][df_corrects[i]['基金統編']==item]['本月定時定額扣款金額'].values[0]
                    sip_money.append(a)
                    sip_moneys[item]=sip_money
    return pd.DataFrame(sip_moneys)

In [76]:
china_sip_money=get_moneys(result_id['China'])
for item in  result_id['China']:
    china_sip_money[item]= china_sip_money[item]/china_sip_money[item].max()
china_sip_money.columns = result_n['China']

In [77]:
brazil_sip_money=get_moneys(result_id['Brazil'])
for item in  result_id['Brazil']:
    brazil_sip_money[item]= brazil_sip_money[item]/brazil_sip_money[item].max()
brazil_sip_money.columns = result_n['Brazil']

In [78]:
russia_sip_money=get_moneys(result_id['Russia'])
for item in  result_id['Russia']:
    russia_sip_money[item]= russia_sip_money[item]/russia_sip_money[item].max()
russia_sip_money.columns = result_n['Russia']
# russia_sip_money

In [79]:
india_sip_money=get_moneys(result_id['India'])
for item in  result_id['India']:
    india_sip_money[item]= india_sip_money[item]/india_sip_money[item].max()
india_sip_money.columns = result_n['India']

In [36]:
df=get_funding(aliveDF)
df_china=df[result_id['China']]
df_china.columns = [result_n['China']]
df_china
print(df_china.to_json(force_ascii=False))
f = open('china.json','w')
f.write(df_china.to_json(force_ascii=False))
f.close()

{"["日盛中國內需動力基金"]":{"0":548,"1":567,"2":575,"3":568,"4":556,"5":545,"6":536,"7":533,"8":525,"9":494,"10":515,"11":511,"12":498,"13":488,"14":498,"15":493,"16":484,"17":487,"18":463,"19":437,"20":425,"21":415,"22":426,"23":420,"24":398,"25":404,"26":394,"27":396,"28":398,"29":408,"30":410,"31":410,"32":409,"33":392,"34":406,"35":395},"["野村中國機會基金"]":{"0":1540,"1":1558,"2":1503,"3":1505,"4":1436,"5":1406,"6":1386,"7":1365,"8":1365,"9":1290,"10":1285,"11":1259,"12":1243,"13":1231,"14":1233,"15":1244,"16":1235,"17":1233,"18":1219,"19":1220,"20":1216,"21":1215,"22":1223,"23":1199,"24":1201,"25":1222,"26":1259,"27":1288,"28":1343,"29":1423,"30":1557,"31":1657,"32":1685,"33":1746,"34":1794,"35":1811},"["第一金中國世紀基金-新臺幣"]":{"0":4285,"1":5539,"2":5556,"3":5523,"4":5377,"5":5200,"6":5233,"7":5188,"8":4886,"9":4139,"10":5181,"11":4698,"12":4622,"13":4598,"14":4557,"15":4614,"16":4635,"17":4633,"18":4521,"19":4423,"20":3845,"21":3896,"22":5240,"23":4301,"24":4269,"25":4285,"26":4155,"27":3982,"28":393

In [37]:
df_india=df[result_id['India']]
df_india.columns = [result_n['India']]
df_india
f = open('india.json','w')
f.write(df_india.to_json(force_ascii=False))
f.close()

In [39]:
df_russia=df[result_id['Russia']]
df_russia.columns = [result_n['Russia']]
df_russia
f = open('russia.json','w')
f.write(df_russia.to_json(force_ascii=False))
f.close()

In [38]:
df_brazil=df[result_id['Brazil']]
df_brazil.columns = [result_n['Brazil']]
f = open('brazil.json','w')
f.write(df_brazil.to_json(force_ascii=False))
f.close()

In [80]:
f = open('russia_percentage.json','w')
f.write(russia_sip_money.to_json(force_ascii=False))
f.close()

In [81]:
f = open('india_percentage.json','w')
f.write(india_sip_money.to_json(force_ascii=False))
f.close()

In [82]:
f = open('brazil_percentage.json','w')
f.write(brazil_sip_money.to_json(force_ascii=False))
f.close()

In [83]:
f = open('china_percentage.json','w')
f.write(china_sip_money.to_json(force_ascii=False))
f.close()

In [ ]:
pd.set_option('display.max_rows', None)
df2 = pd.read_html("https://www.moneydj.com/funddj/yb/YP302000.djhtm?a=ET003001")
df3 = pd.read_html("https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET003002")
df4 = pd.read_html("https://www.moneydj.com/funddj/yb/yp302000.djhtm?a=ET003003")
dfFinal = pd.concat([df2[4], df3[4], df4[4]], axis=0)
dfFinal

In [ ]:
for name in newAliveList:
    filterName = (dfFinal["基金名稱"]["基金名稱"] == name)
    try:
        print(dfFinal[filterName]["報酬率(%)"]["六個月"].values[0])
    except:
        print(name)